<a href="https://colab.research.google.com/github/ykitaguchi77/PreProcess_Image_colab/blob/master/split_dataset_for_crossvalidation_includingtest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Split datasets for cross validation**
Trainデータセットを5分割し、そのうち1つをvalセットに、残りの合計をtestセットに分割するスクリプト


In [112]:
import random
import glob
import os
import shutil
import numpy as np
import time

from google.colab import drive
drive.mount('/content/drive')


'''
-----orig_data-----grav
                |--cont
↓
↓

-----dst_data[0]------dst_train[0]----grav
  |                |               |-- cont
  |                |--dst_val[0]------grav
  |                                |--cont
  |
  |--dst_data[1]------dst_train[1]----grav
  |                |               |-- cont
  |                |--dst_val[1]------grav
  |                                |--cont
  ...
  |--dst_data[9]------dst_train[9]----grav
  |                |               |-- cont
  |                 |--dst_val[9]------grav
  |                                 |--cont
  |
  |--dst_test---grav
              |--cont
'''

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


'\n-----orig_data-----grav\n                |--cont\n↓\n↓\n\n-----dst_data[0]------dst_train[0]----grav\n  |                |               |-- cont\n  |                |--dst_val[0]------grav\n  |                                |--cont\n  |\n  |--dst_data[1]------dst_train[1]----grav\n  |                |               |-- cont\n  |                |--dst_val[1]------grav\n  |                                |--cont\n  ...\n  |--dst_data[9]------dst_train[9]----grav\n  |                |               |-- cont\n  |                 |--dst_val[9]------grav\n  |                                 |--cont\n  |\n  |--dst_test---grav\n              |--cont\n'

#**Module群**

In [113]:
def get_path(orig_path, dst_path, split_num, test_num):
    classes = os.listdir(orig_path) #クラス名を取得
    #データの分割数を設定
    data_list = [0]*len(classes)
    k=0
    for i in classes:
        data_list[k] = glob.glob(orig_path+'/'+i+'/*')
        k+=1
    split_length = int((len(data_list)-test_num)/split_num)
    return data_list, classes, split_length

def makefolder(orig_path, dst_path, classes):
    #フォルダを作成
    if not os.path.exists(dst_path):  # ディレクトリがなかったら
         # 作成したいフォルダ名を作成
        for i in range(split_num):
            for j in classes:
                os.makedirs(dst_path+'/'+str(i)+'/train/'+j, exist_ok=True)
                os.makedirs(dst_path+'/'+str(i)+'/val/'+j, exist_ok=True)
        for j in classes:
            os.makedirs (dst_path+'/test/'+j, exist_ok=True)
            os.makedirs (dst_path+'/test/'+j, exist_ok=True)


def split_testset(data_list, test_num, classes):
    dst_test = [0] * test_num
    test_num_class = []

    #1クラスごとのテストセットの枚数を確認
    test_num_class = int(test_num/len(classes))

    #testsetの指定
    dst_test = random.sample(data_list, test_num_class)

    #データセット全体とtestsetの差分を取り、dst_dataに格納
    dst_data = [x for x in data_list if x not in dst_test]

    return dst_data, dst_test


def split_data_list(data_list, split_num):
    split_data, dst_data, dst_train, dst_val, dst_test = [0]*split_num, [0]*split_num, [0]*split_num, [0]*split_num, [0]*split_num

    #データの分割
    split_data = list(np.array_split(data_list, split_num))

    #データセット全体と分割したデータの差分を取り、dst_dataに格納

    dst_data = [0] * split_num
    for i in range(split_num):
        dst_data[i] = [x for x in data_list if x not in split_data[i]]

    #トレーニングセット、バリデーションセット、バリデーションセットのリスト作成
    for i in range(split_num):
        dst_train[i] = dst_data[i]
        dst_val[i] = split_data[i]  #バリデーションセット

    return dst_train, dst_val


def copy_to_folders(split_num, class_name, dst_train, dst_val, dst_path):
    k=0
    for i in range(split_num):
        dst_path_train = dst_path +'/'+str(i)+'/train/'+class_name
        dst_path_val = dst_path +'/'+str(i)+'/val/'+class_name

        for p in dst_train[k]:  # 選択したファイルを目的フォルダにコピー
            shutil.copy(p, dst_path_train)
            #print(p)
            print(dst_path_train)

        for p in dst_val[k]:  # 選択したファイルを目的フォルダにコピー
            shutil.copy(p, dst_path_val)
            #print(p)    
            print(dst_path_val)
        k+=1

def copy_testset(class_name, dst_test, dst_path):

    dst_path_test = dst_path +'/test/'+class_name

    for p in dst_test:  # 選択したファイルを目的フォルダにコピー
        shutil.copy(p, dst_path_test)
        #print(p)    
        print(dst_path_test)


def main():
    orig_path = "/content/drive/MyDrive/Deep_learning/666mai_dataset/gravcont_250px"
    dst_path = "/content/drive/MyDrive/Deep_learning/666mai_dataset/crossvalidation_250px"  # フォルダ名
    split_num = 5  #データをいくつに分割するかを記載
    test_num = 86  #1クラスごとのtestsetの枚数を記載

    data_list, classes, split_length = get_path(orig_path, dst_path, split_num, test_num)
    makefolder(orig_path, dst_path, classes)

    k=0
    for i in range(len(classes)):
        dst_data, dst_test = split_testset(data_list[k], test_num, classes)
        dst_train, dst_val = split_data_list(dst_data, split_num)
        copy_to_folders(split_num, classes[k], dst_train, dst_val, dst_path)
        copy_testset(classes[k], dst_test, dst_path)
        k+=1

In [ ]:
if __name__ == "__main__":
    start = time.time()
    main()
    elapsed_time = time.time() - start
    print ("elapsed_time:{0}".format(elapsed_time) + "[sec]")

In [ ]:
    orig_path = "/content/drive/MyDrive/Deep_learning/666mai_dataset/gravcont_250px"
    dst_path = "/content/drive/MyDrive/Deep_learning/666mai_dataset/crossvalidation_250px"  # フォルダ名
    split_num = 5  #データをいくつに分割するかを記載
    test_num = 86  #1クラスごとのtestsetの枚数を記載

    data_list, classes, split_length = get_path(orig_path, dst_path, split_num, test_num)
    makefolder(orig_path, dst_path, classes)

    k=0
    for i in range(len(classes)):
        dst_data, dst_test = split_testset(data_list[k], test_num, classes)
        dst_train, dst_val = split_data_list(dst_data, split_num)
        copy_to_folders(split_num, classes[k], dst_train, dst_val, dst_path)
        copy_testset(classes[k], dst_test, dst_path)
        k+=1

#**作ったフォルダを削除したいとき**

In [111]:
dst_path = "/content/drive/MyDrive/Deep_learning/666mai_dataset/crossvalidation_250px"
directory = dst_path
try:
    shutil.rmtree(directory)
except FileNotFoundError:
    pass

#**ファイル数の確認**

In [123]:
import glob

# ファイル数を調べたいフォルダのパス
path = "/content/drive/MyDrive/Deep_learning/666mai_dataset/crossvalidation_250px/2/val/cont/*" 
 
# フォルダ内の全ファイル名をリスト化
files = glob.glob(path)
 
# リストの長さ（ファイル数）を取得
count = len(files)
 
# ファイル数を確認
print(count)

# ファイルの中身を確認
print(files)

58
['/content/drive/MyDrive/Deep_learning/666mai_dataset/crossvalidation_250px/2/val/cont/2139.JPG', '/content/drive/MyDrive/Deep_learning/666mai_dataset/crossvalidation_250px/2/val/cont/4560.jpg', '/content/drive/MyDrive/Deep_learning/666mai_dataset/crossvalidation_250px/2/val/cont/3222.jpg', '/content/drive/MyDrive/Deep_learning/666mai_dataset/crossvalidation_250px/2/val/cont/7402.jpg', '/content/drive/MyDrive/Deep_learning/666mai_dataset/crossvalidation_250px/2/val/cont/444.jpg', '/content/drive/MyDrive/Deep_learning/666mai_dataset/crossvalidation_250px/2/val/cont/4329.jpg', '/content/drive/MyDrive/Deep_learning/666mai_dataset/crossvalidation_250px/2/val/cont/1045.JPG', '/content/drive/MyDrive/Deep_learning/666mai_dataset/crossvalidation_250px/2/val/cont/168.jpg', '/content/drive/MyDrive/Deep_learning/666mai_dataset/crossvalidation_250px/2/val/cont/3902.JPG', '/content/drive/MyDrive/Deep_learning/666mai_dataset/crossvalidation_250px/2/val/cont/2556.JPG', '/content/drive/MyDrive/Deep